In [1]:
import pandas as pd
import statsmodels.api as sm
import numpy as np
import time

C:\Users\natan\Anaconda3\lib\site-packages\statsmodels\compat\pandas.py:49: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  data_klasses = (pandas.Series, pandas.DataFrame, pandas.Panel)


In [340]:
df=pd.read_csv('twitter_data_with_vix_for_analysis.csv',parse_dates=['created_at'],index_col=0)

In [341]:
for x in df.columns:
    print(x)

text
created_at
retweet_count
favorite_count
trade_topic
china_topic
financial_topic
labor_topic
industry_topic
currency_rates_topic
general_related_topic
budget_topic
foreign_policy_topic
covid_topic
sensitive_topic
environment_topic
sentiment
month
1_minutes_forward_pct_change
5_minutes_forward_pct_change
10_minutes_forward_pct_change
15_minutes_forward_pct_change
30_minutes_forward_pct_change
1_hours_forward_pct_change
3_hours_forward_pct_change
6_hours_forward_pct_change
Facebook
Instagram
Media Studio
Mobile Web (M5)
Neatly For BlackBerry 10
Periscope
TweetDeck
TwitLonger Beta
Twitlonger
Twitter Ads
Twitter Media Studio
Twitter Mirror for iPad
Twitter QandA
Twitter Web App
Twitter Web Client
Twitter for Android
Twitter for BlackBerry
Twitter for Websites
Twitter for iPad
Twitter for iPhone
Vine - Make a Scene


In [342]:
X=df.loc[:,['created_at',
'Twitter for iPhone',
'Twitter for Android',
'favorite_count',
'retweet_count',
'trade_topic',
'china_topic',
'financial_topic',
'labor_topic',
'industry_topic',
'currency_rates_topic',
'general_related_topic',
'budget_topic',
'foreign_policy_topic',
'covid_topic',
'sensitive_topic',
'environment_topic',
'sentiment'
]].copy()

In [343]:
y=df[['1_minutes_forward_pct_change',
'5_minutes_forward_pct_change',
'10_minutes_forward_pct_change',
'15_minutes_forward_pct_change',
'30_minutes_forward_pct_change',
'1_hours_forward_pct_change',
'3_hours_forward_pct_change',
'6_hours_forward_pct_change']].copy()
# '1_minutes_forward_vol_sum',
# '5_minutes_forward_vol_sum',
# '10_minutes_forward_vol_sum',
# '15_minutes_forward_vol_sum',
# '30_minutes_forward_vol_sum',
# '1_hours_forward_vol_sum',
# '3_hours_forward_vol_sum']].copy()

In [503]:
inspect_df=pd.concat([X.loc[~y['15_minutes_forward_pct_change'].isna()],y['15_minutes_forward_pct_change'].dropna()],axis=1)

In [504]:
inspect_df=inspect_df.loc[inspect_df['created_at']>='2016-05-03 22:00:00'].copy()

In [505]:
for column in inspect_df.iloc[:,:-1].columns:
    inspect_df[column] = inspect_df[[column, 'sentiment']].product(axis=1)
# inspect_df.drop('sentiment',1,inplace=True)

In [506]:
inspect_df.drop('created_at',1,inplace=True)

In [507]:
# inspect_df.insert(0, 'financial_topic_sentiment_interaction',
#                              inspect_df[["sentiment", 'financial_topic']].product(axis=1))
# inspect_df.insert(0, 'sensitive_topic_sentiment_interaction',
#                              inspect_df[["sentiment", 'sensitive_topic']].product(axis=1))

In [508]:
# inspect_df=inspect_df.replace([np.inf, -np.inf], float('nan'))
inspect_df=inspect_df.dropna()

In [509]:
mod = sm.OLS(inspect_df.iloc[:,-1],inspect_df.iloc[:,:-1])
res = mod.fit()

In [510]:
print(res.summary())

                                  OLS Regression Results                                 
Dep. Variable:     15_minutes_forward_pct_change   R-squared:                       0.006
Model:                                       OLS   Adj. R-squared:                  0.002
Method:                            Least Squares   F-statistic:                     1.494
Date:                           Thu, 18 Jun 2020   Prob (F-statistic):             0.0869
Time:                                   16:14:22   Log-Likelihood:                 13063.
No. Observations:                           4517   AIC:                        -2.609e+04
Df Residuals:                               4500   BIC:                        -2.598e+04
Df Model:                                     17                                         
Covariance Type:                       nonrobust                                         
                            coef    std err          t      P>|t|      [0.025      0.975]
----------

In [177]:
from sklearn.linear_model import Ridge
rr = Ridge(alpha=0.00000000001,normalize=False)
rr.fit(inspect_df.iloc[:,:-1], inspect_df.iloc[:,-1])
w = rr.coef_

In [178]:
print(rr.score(inspect_df.iloc[:,:-1], inspect_df.iloc[:,-1]))

0.0021036162471321784


In [24]:
from sklearn import linear_model
clf = linear_model.Lasso(alpha=0.000000001,normalize=False)
clf.fit(inspect_df.iloc[:,:-1], inspect_df.iloc[:,-1])
print(clf.score(inspect_df.iloc[:,:-1], inspect_df.iloc[:,-1]))

0.009015501826825978


C:\Users\natan\Anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:476: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 0.15760652487771254, tolerance: 0.0001046145706115335
  positive)


In [ ]:
coefs=pd.DataFrame(columns=['variable','coefficients'])

In [ ]:
coefs['variable']=inspect_df.iloc[:,:-1].columns

In [ ]:
coefs['coefficients'] = clf.coef_

In [ ]:
len(coefs[coefs['coefficients']!=0])

In [ ]:
for i,x in coefs[coefs['coefficients']!=0].sort_values('coefficients').iterrows():
    print(x['variable'])

In [267]:
inspect_df

,created_at,Twitter for iPhone,Twitter for Android,favorite_count,retweet_count,trade_topic,china_topic,financial_topic,labor_topic,industry_topic,currency_rates_topic,general_related_topic,budget_topic,foreign_policy_topic,covid_topic,sensitive_topic,environment_topic,sentiment,15_minutes_forward_pct_change
2,2009-05-08 13:38:08,0,0,-4.053026,-3.374929,0.0,0.0,0.043478,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.12125,0.004115
4,2009-05-12 14:07:28,0,0,2.829411,2.785574,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00000,-0.002143
8,2009-05-15 14:13:13,0,0,-4.053026,-4.473542,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00000,0.000916
11,2009-05-18 14:26:00,0,0,-0.131053,-0.379197,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.80000,-0.007785
13,2009-05-20 13:25:39,0,0,-2.261267,-2.864104,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.80000,-0.007289
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26867,2016-11-03 12:34:14,0,1,-0.465678,-0.390794,0.0,0.0,0.000000,0.037037,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.01875,0.036652
26868,2016-11-03 15:19:12,1,0,0.007093,0.267106,0.0,0.0,0.000000,0.064516,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00000,0.003160
26874,2016-11-04 15:32:31,1,0,-1.732074,-1.711859,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00000,0.022292
26920,2016-11-08 11:43:14,0,1,2.057669,2.707292,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.00000,-0.007684
